In [6]:
import os
import pickle
import threading
import time
import gspread
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from decimal import Decimal

from ibapi.client import EClient
from ibapi.wrapper import EWrapper

# Define the OAuth scopes for Sheets and Drive.
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

def get_gspread_client():
    """Perform OAuth flow and return an authorized gspread client."""
    creds = None
    # token.pickle stores the user's credentials after the first login.
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # If there are no valid credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # 'client_secret.json' is the file you downloaded from Google Cloud.
            flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for next time.
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return gspread.authorize(creds)

def update_google_sheet(positions):
    """Update the 'Investments' worksheet with positions data."""
    client = get_gspread_client()
    # Open the spreadsheet by key (from your URL)
    spreadsheet = client.open_by_key("1-qKzKz0iij2RRevzrusoUApofTgQTbIOoK3eOBEwuOc")
    worksheet = spreadsheet.worksheet("Investments")

    # Prepare data with a header row.
    data = [["Symbol", "Exchange", "Quantity", "AvgCost"]]
    for pos in positions:
        # Convert Decimal values to float (if needed)
        symbol = pos.get("Symbol")
        exchange = pos.get("Exchange")
        quantity = pos.get("Quantity")
        avg_cost = pos.get("AvgCost")
        if isinstance(quantity, Decimal):
            quantity = float(quantity)
        if isinstance(avg_cost, Decimal):
            avg_cost = float(avg_cost)
        row = [symbol, exchange, quantity, avg_cost]
        data.append(row)

    # Update the sheet starting at cell A1.
    # Note: Newer versions of gspread require values first then range_name.
    worksheet.update(values=data, range_name="A1")
    print("Google Sheet updated with positions data.")

# IBKR client code.
class IBKRClient(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = []  # List to store position data

    def nextValidId(self, orderId: int):
        print(f"Connected to IBKR. Next valid order ID: {orderId}")
        self.reqPositions()  # Request all open positions

    def position(self, account: str, contract, position: float, avgCost: float):
        symbol = contract.symbol
        sec_type = contract.secType
        exchange = contract.exchange

        self.positions.append({
            "Symbol": symbol,
            "Exchange": exchange,
            "Quantity": position,
            "AvgCost": avgCost,
        })

    def positionEnd(self):
        print("\nAll Positions Retrieved:")
        for pos in self.positions:
            print(pos)
        # Update Google Sheet once positions are retrieved.
        update_google_sheet(self.positions)
        self.disconnect()  # Disconnect after updating

def run_ibkr():
    app = IBKRClient()
    # Connect to IBKR on port 4001.
    app.connect("127.0.0.1", 4001, clientId=1)
    # Run API in a separate thread.
    api_thread = threading.Thread(target=app.run, daemon=True)
    api_thread.start()
    time.sleep(5)  # Wait for data to be retrieved.
    app.disconnect()

if __name__ == "__main__":
    run_ibkr()


ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2107 HMDS data farm connection is inactive but should be available upon demand.ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Connected to IBKR. Next valid order ID: 1

All Positions Retrieved:
{'Symbol': 'ASML', 'Exchange': 'NASDAQ', 'Quantity': Decimal('0.5409'), 'AvgCost': 704.2222222}
{'Symbol': 'MUV2', 'Exchange': 'IBIS', 'Quantity': Decimal('0.3'), 'AvgCost': 396.38333335}
{'Symbol': 'MSTR', 'Exchange': 'NASDAQ', 'Quantity': Decimal('2'), 'AvgCost': 337.3029}
{'Symbol': 'GOOG', 'Exchange': 'NASDAQ', 'Quantity': Decimal('4.5045'), 'AvgCost': 163.84397825}
{'Symbol': 'ARM', 'Exchange': 'NASDAQ', 'Quantity': Decimal('2.5'), 'AvgCost': 86.8722}
{'Symbol': 'AVGO', 'Exchange': 'NASDAQ', 'Quantity': Decimal('1.5088'), 'AvgCost': 132.0473224}
{'Symbol': 'MSFT', 'Exchange': 'NASDAQ', 'Quantity': Decimal('1.5028'), 'AvgCost': 392.32938515}
{'Symbol': 'UNH', 'Exchange': 'NYSE', 'Quantity': Decimal('0.4021'), 'AvgCost': 465.8468043}
{'Symbol': 'YPF', 'Exchange': 'NYSE', 'Quantity': Decimal('8'), 'AvgCost': 26.0012375}
{'Symbol': 'CRWD', 'Exchange': 'NASDAQ', 'Quantity': Decimal('1.2'), 'AvgCost': 262.87608335}
{'Sy